## Process public and relevant datasets

### 1. Imports

In [1]:
import json
import ee
import geemap
import folium ###
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import time
import math ###
import geojson 
from datetime import datetime ###
from sidecar import Sidecar ###

from modules.gee_initialize import initialize_ee 

initialize_ee()

from modules.json_to_ee import json_to_feature_with_id ###
# from modules.agstack_setup import start_agstack_session ###
import modules.agstack_to_gee as agstack_to_gee ###
import modules.area_stats as area_stats
from modules.country_allocation import find_country_from_modal_stats
import modules.tidy_tables as tidy_tables

from parameters import * # NB for run-specific parameters edit "parameters/config_runtime"

from modules.file_to_ceo import get_ceo_url ###
from modules.image_prep import export_image_collection_to_asset, add_multi_lookup_properties_to_image_collection
from modules.image_prep import add_multi_lookup_properties_to_image_collection
from parameters.config_lookups import lookup_gee_datasets

print ("imports complete")

Earth Engine has been initialized with the specified credentials.
imports complete


### 2. Call and process datasets

In [2]:
if use_existing_image_collection:
    images_iCol = ee.ImageCollection(target_image_col_id) 
    print ("using existing image collection asset")
else:
    print ("compiling image collection on the fly...")
    from datasets.combining_datasets import images_iCol

compiling image collection on the fly...
scale (m):  27.829872698318393
scale (m):  10
scale (m):  9.276624232772797
scale (m):  10
scale (m):  10.080003101582253
scale (m):  27.829872698318393
scale (m):  10
scale (m):  27.829872698318393
scale (m):  27.829872698318393
scale (m):  27.829872698318393
scale (m):  27.829872698318393
scale (m):  24.73766462072746
scale (m):  10
scale (m):  30.000000000000004
scale (m):  30.000000000000004
scale (m):  30.000000000000004
scale (m):  27.829872698318393
scale (m):  27.829872698318393
scale (m):  10


EEException: Collection.loadTable: Collection asset 'WCMC/WDPA/current/polygons' not found.

In [ ]:
## get lists from lookup for different streams of processing
from  dataset_properties.make_processing_lists_from_lookup import buffer_stats_list,\
                                                                  presence_only_flag_list,\
                                                                  country_allocation_stats_only_list, \
                                                                  plot_stats_list, \
                                                                  decimal_place_column_list
if debug: print ("processed")

In [ ]:
#filtering to remove images in exclusion list (from parameters.config_runtime), useful if dont want certain datasets
images_iCol_filt = images_iCol.filter(ee.Filter.inList("dataset_id",exclusion_list_dataset_ids).Not())
if debug: print ("excluding dataset ids: ", exclusion_list_dataset_ids)

In [ ]:
if update_iCol_properties ==True or use_existing_image_collection == False:
    images_iCol_w_properties = add_multi_lookup_properties_to_image_collection(images_iCol_filt,"dataset_id",
                                                            lookup_gee_datasets, "dataset_id",
                                                            "dataset_name","system:index",
                                                            "dataset_order","dataset_order",
                                                            "country_allocation_stats_only","country_allocation_stats_only")
    images_iCol_filt = images_iCol_w_properties
    if debug: print ("properties added/updated")